In [45]:
from bs4 import BeautifulSoup
import urllib.request,sys,time
import requests
import pandas as pd
import re
import math
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from datetime import datetime

#### Loop titlesAndLinks is empty (len = 0)

In [69]:
titles_list = list()
links_list = list()
dates_list = list()

# Loop through url by incrementing start number by 10, each page has 10 results
for page in range(0,200,10):
    url = 'https://www.fedsearch.org/board_public/search?start={num}&number=10&source=board_pub&text=cloud+advisory&submit=Search'.format(num=page)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, "html.parser")
    results = soup.find('div', attrs={'id':'results'})
    titlesAndLinks = results.find_all('a',href=True)
    
    # Checking if there are any more results
    if int(len(titlesAndLinks)) == 0:
        break
        
    # Counter to alternate appending to title and link
    count = 1
    for a in titlesAndLinks:
        if count % 2 == 0:
            links_list.append(a.text.strip())
        else:
            titles_list.append(a.text.strip())
        count+=1
        

for link in links_list:
    url = link
    if url.__contains__('pdf'):
        dates_list.append('-')
        continue
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.text, "html.parser")
    except Exception as e:
        error_type, error_obj, error_info = sys.exc_info()
        print ('ERROR FOR LINK:',url)                       
        print (error_type, 'Line:', error_info.tb_lineno)
        dates_list.append('-')
        continue

    date = soup.find('div', class_='lastUpdate')
    if date is None:
        dates_list.append('-')
        continue
    else:
        dates_list.append(date.text.strip())


d = {'Title': titles_list, 'Link': links_list, 'Date':dates_list}
df = pd.DataFrame(data = d)

# Importing to csv
df.to_csv(r'C:\Users\65936\\OneDrive\Desktop\MAS\\USFED.csv')

#### Below is a working tool to scrape recent postings, but only shows latest 35 postings, and does not go through entire month

In [39]:
url = 'https://www.federalreserve.gov/recentpostings.htm'
page = requests.get(url)
soup = BeautifulSoup(page.text, "html.parser")
titlesAndLinks = soup.find_all('div', class_='col-xs-9 col-md-10 eventlist__event')
dates = soup.find_all('div', class_='col-xs-3 col-md-2 eventlist__time')

links_list = list()
titles_list = list()
dates_list = list()

base_url = 'https://www.federalreserve.gov'


for a in titlesAndLinks:
    new_link = base_url + a.find('a',href=True)['href']
    links_list.append(new_link) # Link
    titles_list.append(a.text.strip()) # Desc
    
for b in dates:
    dates_list.append(b.find('time').text.strip()) # Date


d = {'Title': titles_list,'Link':links_list,'Date':dates_list}
df = pd.DataFrame(data=d)
df.to_csv(r'C:\Users\65936\\OneDrive\Desktop\MAS\\USFED_new.csv')

In [44]:

driver = webdriver.Chrome()
driver.get('https://www.federalreserve.gov/publications/filter.htm')
button = driver.find_element(By.XPATH,"//*[text()=' Submit']")
button.click()
driver.implicitly_wait(5)
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
blocks = soup.find_all('pw-filter-item')
print(blocks)


[]


In [64]:
links = list()
titles= list()
dates=list()

url = 'https://www.federalreserve.gov/recentpostings.htm'
try:
    page = requests.get(url)
except:
    print("Something is wrong with US FED link.")
soup = BeautifulSoup(page.text, "html.parser")
titlesAndLinks = soup.find_all('div', class_='col-xs-9 col-md-10 eventlist__event')
datesBlock = soup.find_all('div', class_='col-xs-3 col-md-2 eventlist__time')

titles.append('US FED')
links.append('US FED')
dates.append('US FED')

base_url = 'https://www.federalreserve.gov'

links_list = list()
titles_list = list()
dates_list = list()

for a in titlesAndLinks:
    new_link = base_url + a.find('a',href=True)['href']
    links_list.append(new_link) # Link
    titles_list.append(a.text.strip()) # Desc


pattern = r'(\d{1,2}/\d{1,2}/2023)' 

for b in datesBlock:
    d = b.find('time').text.strip()
    match = re.search(pattern, d)
    matched_date = match.group(1)
    month, day, year = map(int, matched_date.split('/'))                # Using regex to filter out date. Since date on website uses numerical date instead of string.
    if month == 7 and year == 2023:
            dates_list.append(matched_date)
            dates.append(matched_date)

for a in links_list[0:len(dates_list)]:
    links.append(a)
for b in titles_list[0:len(dates_list)]:
    titles.append(b)

In [67]:
print(links)

['US FED', 'https://www.federalreserve.gov/releases/g5/current/default.htm']


In [55]:
test_list=['7/*/2023']    
temp = '(?:% s)' % '|'.join(test_list)
if re.match(temp, '7/23/2023'):
    print(true)